In [1]:
import pandas as pd
# import re
from datetime import datetime
import string

In [2]:
# Correção para linhas que começam com múltiplos espaços

def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)

# Função para separar colunas do df importado

def separar_colunas_df(df):
    # lista alfabetica e.g ['a','b','c'...]
    lista_alfabeto = list(string.ascii_lowercase)
    # Pega a primeira coluna do df (que tem todos os dados) e substitui onde possui mais de um espaço por | 
    df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)
    # corrige a substituição para linhas que começam com espaços
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
    # corta a partir do separador '|' e define cada pedaço para uma coluna ['a','b'..]
    for i in range (len(df.iloc[:,0][0].split('|'))):
        df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
    # deleta a coluna que originou as demais colunas    
    df.drop(df.columns[0], axis = 1, inplace = True)
    return df

# veic_2019_02.csv_amostra

In [3]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]

df_veic = pd.read_csv('veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(4)

,index_df,x,y,data-hora,data,dia,mes,ano,tempo,hora,...,c,d,e,f,placa,h,i,j,k,l
0,index,n,x,tipo,data,dia,mes,ano,hora,y,...,vel2,k,s,cod,placa,r,t,y,u,i
1,1,501,1,2019-02-01 00:00:07,2019-02-01,01,02,2019,00:00:07,00,...,68.0,2,3.8,bdac468f1,LRV5999,0,0,NaN,NaN,NaN
2,2,502,2,2019-02-01 00:00:09,2019-02-01,01,02,2019,00:00:09,00,...,57.0,2,3.8,bdac469a2,LRW4982,0,0,NaN,NaN,NaN
3,3,502,2,2019-02-01 00:00:17,2019-02-01,01,02,2019,00:00:17,00,...,61.0,2,3.6,bdac46ee2,LQM9654,0,0,NaN,NaN,NaN


# movimento_OCR_JAN_2019.txt_amostra.txt

In [4]:
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr_raw = pd.read_csv('movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

df_ocr_split = separar_colunas_df(df_ocr_raw.copy())
df_ocr_split.head()

,a,b,c,d,e
0,KPV68771AUTOMÓVEL,20190101000000971024,0101201900000003900CET8-5455 0410191112025201...,St R Catete,B-22.93380-43.18294
1,LQR82471AUTOMÓVEL,20190101000001181044,0101201900000103400CET8-5410 0410121122025201...,St Tanque,D-22.97019-43.41377
2,ETP18901AUTOMÓVEL,20190101000001281014,0101201900000104000CET8-5436 0410241111025201...,St Cidade Nova,A-22.93149-43.20888
3,KXL99161AUTOMÓVEL,20190101000001381014,0101201900000102500CET8-5439 0410161111025201...,St Colonia,A-22.92479-43.38907
4,LPU22871AUTOMÓVEL,20190101000001901024,0101201900000104400CET8-5401 0410381122025201...,St Pontal,B-23.01267-43.38900


# Tratando OCR

In [5]:
def get_datahora(string):
    date = string[38:52]
    try: 
        date = datetime.strptime(date, "%Y%m%d%H%M%S")
    except:
        date = 'erro'
    return date

def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_placa(string):
    placa = string[0:7]
    return placa.strip()

df_ocr = pd.DataFrame()
df_ocr['CODCET'] = df_ocr_raw['x'].apply(lambda x: x[108:118])
df_ocr['DATAHORA'] = df_ocr_raw['x'].apply(lambda x: get_datahora(x))
df_ocr['PLACA'] = df_ocr_raw['x'].apply(lambda x: get_placa(x))
df_ocr['VELOCIDADE'] = df_ocr_raw['x'].apply(lambda x: get_velocidade(x))
df_ocr['CGT'] = df_ocr_raw['x'].apply(lambda x: x[108:111])
df_ocr['LATITUDE'] = df_ocr_raw['x'].apply(lambda x: x[216:225])
df_ocr['LONGITUDE'] = df_ocr_raw['x'].apply(lambda x: x[225:234])

# Salvando em csv
df_ocr.to_csv('amostra_movimento_tratada.csv', sep=';', index=False)
df_ocr.sample(5)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
84,0410021111,2019-01-01 00:00:38,,32,041,-22.79752,-43.19290
99,0410311111,2019-01-01 00:00:46,,57,041,-22.88907,-43.56765
61,0410151112,2019-01-01 00:00:25,KPA6853,33,041,-22.94040,-43.36801
82,0410301111,2019-01-01 00:00:36,LSN6287,30,041,-22.91483,-43.54121
29,0410552121,2019-01-01 00:00:10,KRR1791,31,041,-23.00726,-43.43819


# SCP_TRANSCARIOCA_0222016_201903_amostra.txt

In [6]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_transcarioca_raw = pd.read_csv('SCP_TRANSCARIOCA_0222016_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas
df_transcarioca_split = separar_colunas_df(df_transcarioca_raw.copy())
df_transcarioca_split.head(5)

,a,b,c,d,e,f
0,-------2carro de passeio,010320190010319,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
1,-------1moto,0103201900104424,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
2,-------2carro de passeio,0103201900104419,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
3,-------1moto,0103201900104628,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
4,-------1moto,0103201900104830,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"


# Tratamento SCP_TRANSCARIOCA

In [7]:
def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_velocidade(string):
    vel = string[92:94]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_latitude(string):
    lat = string[216:219]+"."+string[220:225]
    return lat

def get_longitude(string):
    lon = string[225:228]+"."+string[229:]
    return lon

df_transcarioca = pd.DataFrame()
df_transcarioca['CODCET'] = df_transcarioca_raw['x'].apply(lambda x:x[108:118])
df_transcarioca['DATAHORA'] = df_transcarioca_raw['x'].apply(lambda x: get_datahora(x))
df_transcarioca['PLACA'] = df_transcarioca_raw['x'].apply(lambda x: get_placa(x))
df_transcarioca['VELOCIDADE'] = df_transcarioca_raw['x'].apply(lambda x: get_velocidade(x))
df_transcarioca['CGCT'] = df_transcarioca_raw['x'].apply(lambda x:x[108:111])
df_transcarioca['LATITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_latitude(x))
df_transcarioca['LONGITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_longitude(x))

# Salvando em csv
df_transcarioca.to_csv('amostra_transcarioca_tratada.csv', sep=';', index=False)
df_transcarioca.head(20)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGCT,LATITUDE,LONGITUDE
0,0440021120,2019-03-01 00:10:31,,9,044,-22.87280,-43.33629
1,0440021120,2019-03-01 00:10:44,,24,044,-22.87280,-43.33629
2,0440021120,2019-03-01 00:10:44,,19,044,-22.87280,-43.33629
3,0440021120,2019-03-01 00:10:46,,28,044,-22.87280,-43.33629
4,0440021120,2019-03-01 00:10:48,,30,044,-22.87280,-43.33629
5,0440021120,2019-03-01 00:10:50,,30,044,-22.87280,-43.33629
6,0440021120,2019-03-01 00:10:58,,34,044,-22.87280,-43.33629
7,0440021120,2019-03-01 00:11:07,,32,044,-22.87280,-43.33629
8,0440021120,2019-03-01 00:11:08,,33,044,-22.87280,-43.33629
9,0440021120,2019-03-01 00:11:11,,34,044,-22.87280,-43.33629


# SPLICE_007201820190205.txt_amostra.txt

In [8]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df = pd.read_csv('SPLICE_0072018_20190205_.txt_amostra.txt', sep='|', names = colnames)
# df_splice_raw
# Separando as colunas manualmente

# df_splice_split = separar_colunas_df(df_splice_raw)

# def separar_colunas_df(df):
lista_alfabeto = list(string.ascii_lowercase)

df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)

lista_e = []
df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
for i in range(len(df)):
    lista_e.append(len(df.iloc[:,0][i].split('|'))==4)
#     print(len(df.iloc[:,0][i].split('|')))

# df.iloc[3][0][78:]

a = df.x.where(lista_e)
print(a.dropna().iloc[0])
print(len(a.dropna().iloc[0]))
print(df.iloc[0][0])
print(len(df.iloc[0][0]))
# print(df.shape)
# print(len(lista_e))
# print(df.iloc[:,0][0].split('|'))
# for i in range (len(df.iloc[:,0][0].split('|'))):
#     try:
#         df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
#     except:
#         df[lista_alfabeto[i]] = 'erro'
# df.drop(df.columns[0], axis = 1, inplace = True)

# df

# df_splice['x'] = df_splice.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
# df_splice['x'] = df_splice.x.apply(lambda x: correctSplit(x))
# df_splice['veiculo'] = df_splice.x.apply(lambda x: x.split('|')[0])
# df_splice['a'] = df_splice.x.apply(lambda x: x.split('|')[1])
# df_splice['b'] = df_splice.x.apply(lambda x: x.split('|')[2])
# df_splice['c'] = df_splice.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
# df_splice.drop('x', axis = 1, inplace = True)

# df_splice_split.sample(5)

1MOTO|2019010100004705300RD3399|04505612120072018AV PRES ANTONIO CARLOS PX ED MENEZES CORTES PLAT-ST CANDELARCandelaria|2-22.90751-43.17307
139
1MOTO|2019010100000105900RD3389|04503811120072018AV GAL SAN MARTIN PX 220|Sao Conrado|2-22.98468-43.21814
105


In [9]:
i = 0
print(df.iloc[i][0][78:], end = '\n\n')
print(df.iloc[i][0][30:], end = '\n\n')
print(df.iloc[i][0][140:], end = '\n\n')

Conrado|2-22.98468-43.21814

9|04503811120072018AV GAL SAN MARTIN PX 220|Sao Conrado|2-22.98468-43.21814





# FISCALTECH_0042016_06_2019.txt_amostra.txt

In [10]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_fiscaltech_raw = pd.read_csv('FISCALTECH_0042016_06_2019.txt_amostra.txt', sep='|', names = colnames)
df_fiscaltech = separar_colunas_df(df_fiscaltech_raw)


# Extraindo partes onde possui informação requisitada
df_fiscaltech['b0'] = df_fiscaltech.b
df_fiscaltech['d'] = df_fiscaltech.c # aqui existe latidude e longitude, além de um número prévio que também poderia sar o 
#código do tipo
df_fiscaltech['b'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[0])
df_fiscaltech['c'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[2][:17])

df_fiscaltech.head(4)

,a,b,c,b0,d
0,1,0106201903270901800FSCII2175,04200611130042016,0106201903270901800FSCII2175 0420061113004201...,3-23000409-43334161
1,1,0106201901164305100FSCII2214,04201712110042016,0106201901164305100FSCII2214 0420171211004201...,1
2,1,0106201901120105000FSCII2174,04200711240042016,0106201901120105000FSCII2174 0420071124004201...,4-23001615-43328645
3,1,0106201903304702100FSCII2175,04200611140042016,0106201903304702100FSCII2175 0420061114004201...,4-23000409-43334161


# Tratamento FISCALTECH

In [11]:
def get_datahora(string):
    date = string[:13]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date


def get_lat(string):
    if len(string.split('-')) == 1: #se estamos no brasil, Lat e Long são <0. Então esses campos estão preenchidos, há '-'
        return ''
    else:
        return '-'+string.split('-')[1][:2]+'.'+string.split('-')[1][2:]

def get_long(string):
    if len(string.split('-')) == 1: #se estamos no brasil, Lat e Long são <0. Então esses campos estão preenchidos, há '-'
        return ''
    else:
        return '-'+string.split('-')[2][:2]+'.'+string.split('-')[2][2:]

def get_placa(string):
    if len(string)==1:
        return ''
    else:
        return string[:7]

def get_tipo(string):
    if len(string)==1:
        return string
    else:
        return string[7:8]


#Separação de colunas 

df_fiscaltech['PLACA'] = df_fiscaltech.a.apply(lambda x: get_placa(x))
df_fiscaltech['TIPO'] = df_fiscaltech.a.apply(lambda x: get_tipo(x))
df_fiscaltech['CODCET'] = df_fiscaltech.c.apply(lambda x: x[:10])
df_fiscaltech['CGT'] = df_fiscaltech.c.apply(lambda x: x[:3])
df_fiscaltech['DATAHORA'] = df_fiscaltech.b.apply(lambda x: get_datahora(x))
df_fiscaltech['VELOCIDADE'] = df_fiscaltech.b.apply(lambda x: x[15:17]) #o certo seria [14:19], mas estamos padronizando
#sem casa decimal
df_fiscaltech['LATITUDE'] = df_fiscaltech.d.apply(lambda x: get_lat(x))
df_fiscaltech['LONGITUDE'] = df_fiscaltech.d.apply(lambda x: get_long(x)) #perceber que deixo um número antes do - de fora

df_fiscaltech.drop(['a','b','c','d','b0'], axis = 1, inplace = True)

# Salvando em csv
df_fiscaltech.to_csv('amostra_fiscaltech_tratada.csv', sep=';', index=False)

# Print
df_fiscaltech.sample(10)


,PLACA,TIPO,CODCET,CGT,DATAHORA,VELOCIDADE,LATITUDE,LONGITUDE
76,,1,0420081112,042,2019-06-01 06:19:02,15,-23.00303,-43.32414
0,,1,0420061113,042,2019-06-01 03:27:00,18,-23.000409,-43.334161
91,,1,0420061112,042,2019-06-01 06:20:02,32,-23.000409,-43.334161
3,,1,0420061114,042,2019-06-01 03:30:04,21,-23.000409,-43.334161
71,,1,0420081112,042,2019-06-01 03:17:05,15,-23.00303,-43.32414
7,,1,0420081122,042,2019-06-01 01:17:00,47,-23.003226,-43.324684
96,,1,0420141113,042,2019-06-01 03:36:04,53,-22.981353,-43.409694
62,,1,0420081122,042,2019-06-01 01:17:00,43,-23.003226,-43.324684
22,,1,0420101211,042,2019-06-01 01:13:02,12,-23.003679,-43.322327
43,,1,0420161221,042,2019-06-01 03:19:00,32,,


# SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt

In [18]:

#Tratamento da SITRAN
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_sitran_raw = pd.read_csv('SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt', sep='|', names = colnames)

df_sitran_split = separar_colunas_df(df_sitran_raw.copy())
df_sitran_split.head()



,a,b,c,d,e,f
0,-------1moto,0103201910263053,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"1-22,90393-43,27340"
1,-------1moto,0103201910263141,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"2-22,90393-43,27340"
2,-------1moto,0103201910263449,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"3-22,90393-43,27340"
3,LQS56581moto,0103201910490948,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"1-22,90393-43,27340"
4,FLT20441moto,0103201910491046,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"3-22,90393-43,27340"


In [19]:
def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
        
    return date

def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel


def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

df_sitran = pd.DataFrame()
df_sitran['CODCET'] = df_sitran_raw['x'].apply(lambda x: x[108:118])
df_sitran['DATAHORA'] = df_sitran_raw['x'].apply(lambda x: get_datahora(x))
df_sitran['PLACA'] = df_sitran_raw['x'].apply(lambda x: get_placa(x))
df_sitran['VELOCIDADE'] = df_sitran_raw['x'].apply(lambda x: get_velocidade(x))
df_sitran['CGT'] = df_sitran_raw['x'].apply(lambda x: x[108:111])
df_sitran['LATITUDE'] = df_sitran_raw['x'].apply(lambda x: x[216:225])
df_sitran['LONGITUDE'] = df_sitran_raw['x'].apply(lambda x: x[225:234])

# Salvando em csv
df_sitran.to_csv('amostra_sitran_tratada.csv', sep=';', index=False)
df_sitran.sample(20)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
29,0400141110,2019-03-01 10:50:55,,54,040,"-22,90393","-43,27340"
87,0400141110,2019-03-01 10:54:00,,40,040,"-22,90393","-43,27340"
16,0400141110,2019-03-01 10:49:55,CAJ8184,48,040,"-22,90393","-43,27340"
99,0400141110,2019-03-01 10:54:37,,40,040,"-22,90393","-43,27340"
2,0400141110,2019-03-01 10:26:34,,49,040,"-22,90393","-43,27340"
12,0400141110,2019-03-01 10:49:40,,50,040,"-22,90393","-43,27340"
24,0400141110,2019-03-01 10:50:21,,42,040,"-22,90393","-43,27340"
84,0400141110,2019-03-01 10:53:57,KEG1368,37,040,"-22,90393","-43,27340"
54,0400141110,2019-03-01 10:52:18,LMN2038,46,040,"-22,90393","-43,27340"
60,0400141110,2019-03-01 10:52:31,LON3713,44,040,"-22,90393","-43,27340"


# SCP_INTVIA_0182018_201901_amostra.txt

In [13]:
#Tratamento de INTVIA
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_intvia_raw = pd.read_csv('SCP_INTVIA_0182018_201901_amostra.txt', sep='|', names = colnames)

# separando as colunas
df_intvia_split = separar_colunas_df(df_intvia_raw)

df_intvia_split.head(20)



,a,b,c,d,e,f
0,-------1moto,0101201900000215,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
1,LSY84841moto,0101201900000417,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"2-22,96635-43,21941"
2,-------1moto,0101201900000714,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
3,-------1moto,0101201900002229,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
4,-------1moto,0101201900004443,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
5,-------1moto,0101201900004843,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
6,-------1moto,0101201900005239,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"2-22,96635-43,21941"
7,-------1moto,0101201900005442,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
8,-------1moto,0101201900005947,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
9,-------1moto,0101201900010148,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"2-22,96635-43,21941"


In [16]:
def get_datahora(string):
    date = string[0:14]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date
def get_velocidade(string):
    vel = string[14:16]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel
def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa


df_intvia = pd.DataFrame()
df_intvia['CODCET'] = df_intvia_raw['d'].apply(lambda x: x[0:10])
df_intvia['DATAHORA'] = df_intvia_raw['b'].apply(lambda x: get_datahora(x))
df_intvia['PLACA'] = df_intvia_raw['a'].apply(lambda x: get_placa(x))
df_intvia['VELOCIDADE'] = df_intvia_raw['b'].apply(lambda x: get_velocidade(x))
df_intvia['CGT'] = df_intvia_raw['d'].apply(lambda x: x[0:3])
df_intvia['LATITUDE'] = df_intvia_raw['f'].apply(lambda x: x[1:10])
df_intvia['LONGITUDE'] = df_intvia_raw['f'].apply(lambda x: x[10:19])

# Salvando em csv
df_intvia.to_csv('amostra_intvia_tratada.csv', sep=';', index=False)
df_intvia.sample(20)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
19,0480091110,2019-01-01 00:04:30,,42,048,"-22,96635","-43,21941"
71,0480091110,2019-01-01 00:18:56,,45,048,"-22,96635","-43,21941"
16,0480091110,2019-01-01 00:03:40,,26,048,"-22,96635","-43,21941"
47,0480091110,2019-01-01 00:10:51,,49,048,"-22,96635","-43,21941"
81,0480091110,2019-01-01 00:20:37,,53,048,"-22,96635","-43,21941"
30,0480091110,2019-01-01 00:06:33,,44,048,"-22,96635","-43,21941"
94,0480091110,2019-01-01 00:26:18,,13,048,"-22,96635","-43,21941"
32,0480091110,2019-01-01 00:06:40,,48,048,"-22,96635","-43,21941"
42,0480091110,2019-01-01 00:10:29,,50,048,"-22,96635","-43,21941"
75,0480091110,2019-01-01 00:20:15,KOB2899,34,048,"-22,96635","-43,21941"
